# Using `mcpconn` with OpenAI in Google Colab

This notebook demonstrates how to use the `mcpconn` library to interact with OpenAI's API. We will cover:

1. Installing the necessary libraries.
2. Setting up your OpenAI API key in Colab.
3. Making a basic query to an OpenAI model.
4. Managing conversation state to have contextual conversations.
5. Using built-in guardrails to protect your application.

## 1. Installation

First, we need to install `mcpconn` and the `openai` library.

In [1]:
!pip install mcpconn openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00


## 2. Setup OpenAI API Key

To use OpenAI models, you need an API key. `mcpconn` will automatically look for the `OPENAI_API_KEY` in your environment variables.

In Google Colab, you should set this using the **Secrets** tab in the left-hand sidebar.

1. Click on the key icon (🔑) in the left sidebar.
2. Add a new secret with the name `OPENAI_API_KEY`.
3. Paste your OpenAI API key into the value field.
4. Make sure the "Notebook access" toggle is enabled.

## 3. Basic Query

Now, let's make a simple query to OpenAI. We will instantiate `mcpconn` for the `openai` provider and call `connect()` to prepare the client.

In [2]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [7]:
import asyncio
import os
from mclpclient import mcpconn

# In Colab, we need to handle the asyncio event loop specially.
try:
  import nest_asyncio
  nest_asyncio.apply()
except ImportError:
  !pip install nest_asyncio
  import nest_asyncio
  nest_asyncio.apply()

MCP_SERVER_URL="https://mcp.deepwiki.com/mcp"


async def run_basic_query():
    # mcpconn will automatically pick up the OPENAI_API_KEY from the environment.
    client = None
    try:
        client = mcpconn(llm_provider="openai", ssl_verify=False)

        # With the OpenAI provider, we must call connect() to initialize the client state,
        # even if we are not using any MCP tool servers.
        # We can provide a placeholder URL.
        await client.connect(MCP_SERVER_URL)
        print("Client connected.")

        print("\nSending a basic query to OpenAI...")
        response = await client.query("can you provide tools you have")
        print(f"OpenAI: {response}")

    except Exception as e:
        print(f"An error occurred: {e}")
        print("Please ensure your OPENAI_API_KEY is set correctly in Colab Secrets.")
    finally:
        if client and client.is_connected():
            await client.disconnect()
            print("\nClient disconnected.")

# Run the async main function
asyncio.run(run_basic_query())

Client connected.

Sending a basic query to OpenAI...
OpenAI: Absolutely! Here are the main tools and capabilities I can use to help you:

### 1. **Image Analysis**
- I can analyze and interpret images you upload.
- This includes text extraction (OCR), object recognition, diagrams, and more.

### 2. **GitHub Documentation & Code Assistance**
- I can fetch, read, and answer questions about documentation or code from public GitHub repositories.
- This is useful for exploring project structure, usage, setup, and technical questions.

### 3. **General Knowledge and Reasoning**
- I have access to knowledge up until June 2024.
- I can provide explanations, how-to guides, and advice on a wide range of subjects.

### 4. **Text Processing**
- I can summarize, paraphrase, or translate text.
- I can help with writing, debugging, or refactoring code.

If you have a specific task or type of tool in mind, let me know and I can tell you if I provide that service!

Client disconnected.


## 4. Managing Conversation State

`mcpconn` makes it easy to manage conversations. The client can maintain the history of a conversation, allowing the LLM to have context for follow-up questions.

Here's how to start a conversation and have a multi-turn dialogue.

In [9]:
import asyncio
import os
from mclpclient import mcpconn
import nest_asyncio
nest_asyncio.apply()

async def run_conversation():
    client = None
    try:
        client = mcpconn(llm_provider="openai", ssl_verify=False)
        await client.connect(MCP_SERVER_URL)

        # Start a new conversation. This creates a unique ID for the conversation.
        conversation_id = client.start_conversation()
        print(f"Started new conversation: {conversation_id}")

        print("\nQuery 1:")
        response1 = await client.query("can you provide readme of https://github.com/meta-llama/llama")
        print(f"AI: {response1}")

        # The second query will have the context of the first one.
        print("\nQuery 2:")
        response2 = await client.query("how many models are there")
        print(f"AI: {response2}")

        # You can inspect the history at any time.
        history = client.get_conversation_history()
        print("\n--- Conversation History ---")
        for message in history:
            print(f"- {message['role']}: {message['content']}")
        print("--------------------------")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        if client and client.is_connected():
            await client.disconnect()

asyncio.run(run_conversation())

Started new conversation: 80e89a55-ae8d-4de7-b4eb-2f95041f1c98

Query 1:
AI: Here is the **README.md** for [meta-llama/llama](https://github.com/meta-llama/llama) as summarized from the repository:

---

# Llama 2

This repository provides minimal code and utilities to load Meta's Llama 2 large language models (LLMs) and run inference. If you're looking for more detailed examples, particularly using Hugging Face, check out the [llama-cookbook](https://github.com/facebookresearch/llama-recipes/).

> **As of the Llama 3.1 release, this repository is deprecated in favor of a consolidated ecosystem of repositories with expanded functionality.**

## What is Llama 2?

Llama 2 is a family of large language models ranging from 7 billion to 70 billion parameters, developed by Meta. This repository contains:
- Pretrained models: base models for general tasks.
- Fine-tuned chat models: optimized for dialogue and chat use.

Model sizes available:
| Model Size | Parameters | Model Parallel Value | 